**<font  size=6>Hash算法</font>**

**<font  size=5>1. 概念</font>**  
將任意長度的二進制值串映射為固定長度的二進制值串，這個映射的規則就是哈希算法。  
通過原始數據映射之後得到的二進制值串就是哈希值。

**<font  size=5>2. 要求</font>**  
- 從哈希值不能反向推導出原始數據
- 對輸入數據非常敏感，一個Bit 修改得到的哈希值也大不相同
- 散列衝突的概率要很小
- 執行效率高效

**<font  size=5>3. 常見應用</font>**    
安全加密、唯一標識、數據校驗、散列函數、負載均衡、數據分片、分佈式存儲。  
後三個應用均與分佈式系統有關。  
**下面逐個進行解釋。**



**<font  size=5>3.1 安全加密</font>**    

**常用：**
- MD5 (MD5 Message-Digest Algorithm , MD5 消息摘要算法)  
- SHA (Secure HashAlgorithm ,安全散列算法)

**其他：**
- DES ( Data Encryption Standard ,數據加密標準)
- AES ( Advanced Encryption Standard ,高級加密標準)  

哈希算法在安全加密方面有兩點特別重要：

- 很難反向推到
- 散列衝突概率小  

以MD5為例，哈希值是固定的128 位二進制串，最多能表示2^128 個數據。當我們對2^128+1個數據求哈希值時，必然會存在至少兩個哈希值是相同的。

儘管如此，想要破解的難度依然非常高。

**<font  size=5>3.2 唯一標識</font>**    

對大數據做信息摘要，通過一個較短的二進制編碼來表示很大的數據。  

查找圖庫中是否存在某張圖片為例。  
從圖片開頭、中間、末尾各取100 個字節，將這300 個字節放到一塊，通過哈希算法（比如MD5 ），得到一個哈希字符串，用它作為圖片的唯一標識。  
通過唯一標識來判定圖片是否在圖庫中，這樣就可以減少很多工作量。  
還可以把圖片路徑信息，存儲在散列表中。  
如果存在，通過散列表中存儲的文件路徑，獲取到這個已經存在的圖片。  
跟現在要插入的圖片做全量的比對，看是否完全一樣。  
如果一樣，就說明已經存在；  
如果不一樣，說明兩張圖片儘管唯一標識相同，但是並不是相同的圖片。  


**<font  size=5>3.3 數據校驗</font>**    

校驗數據的完整性和正確性。

BT下載時，文件可能分成很多文件塊，並行的下載完成後需要將文件塊組裝。  
組裝時需要校驗文件塊是否完整或是否正確，避免下載出錯或被惡意篡改。

一種校驗思路：  
對所有文件塊取哈希值，保存在種子中。  
下載完成後，逐一對文件塊的哈希值進行比對。  
如果不同，則需要重新從其他宿主機器上下載這個文件塊。  

**<font  size=5>3.4 散列函數</font>**    

散列函數其實也是哈希算法的一種應用。  
散列函數是設計一個散列表的關鍵。  
散列函數中用到的散列算法，更加關注散列後的值是否能平均分佈，以及執行效率。  

**插個問題：如何防止數據庫中的用戶信息被脫庫**  
對用戶密碼進行加密之後再存儲，比如SHA等（因為MD5 已經號稱被破解了）。

但由於很多人使用簡單密碼，為了更加安全，需要維護一個常用密碼的字典表。  
把字典中的每個密碼用哈希算法計算哈希值，然後拿哈希值跟脫庫後的密文比對。  
如果相同，基本上就可以認為，這個加密之後的密碼對應的明文就是字典中的這個密碼。  
（但由於散列衝突，也有可能出現，儘管密文一樣，但是明文並不一樣的情況。）  

引入一個鹽（ salt ），跟用戶的密碼組合在一起，增加密碼的複雜度。


**<font  size=5>3.5 負載均衡</font>**  

在同一個客戶端上，在一次會話中的所有請求都路由到同一個服務器上。

對客戶端IP 地址或者會話ID 計算哈希值，將取得的哈希值與服務器列表的大小進行取模運算，最終得到的值就是應該被路由到的服務器編號。

這樣，我們就可以把同一個IP 過來的所有請求，都路由到同一個後端服務器上。

**<font  size=5>3.6 數據分片 </font>**  

兩個例子來說明。

**統計“ 搜索關鍵詞” 出現的次數**  
假如有1T 日誌文件，記錄了用戶的搜索關鍵詞，想要快速統計出每個關鍵詞被搜索的次數。  
可以先對數據進行分片，然後採用多台機器處理來提高速度。  
依次讀出每個關鍵詞，通過哈希函數計算哈希值，然後跟n取模，得到的值就是應該被分配到的機器編號。  
哈希值相同的搜索關鍵詞就被分配到了同一個機器上。每個機器會分別計算關鍵詞出現的次數，最後合併起來就是最終的結果。  
這裡的處理過程也是MapReduce 的基本設計思想。  

**快速判斷圖片是否在圖庫中**  
上面唯一標識中講過，給每個圖片取唯一標識(或者信息摘要)，然後構建散列表。  
但圖庫中的圖片數量非常大（上億）時，內存有限，單台機器構建散列表行不通。  
可以對數據進行分片，然後採用多機處理。每台機器只維護部分圖片對應的散列表。  
每次從圖庫中讀取一個圖片，計算唯一標識，與機器個數n 求餘取模，得到的值就對應要分配的機器編號，然後將這個圖片的唯一標識和圖片路徑發往對應的機器構建散列表。    
判斷一個圖片是否在圖庫中時，通過相同哈希算法，計算這個圖片的唯一標識，然後與機器個數n 求餘取模。得到的值對應機器編號，將這個圖片的唯一標識和圖片路徑發往對應的機器構建散列表。  

下面估算一億張圖片構建散列表所需大概的機器數。  

假設通過MD5 來計算哈希值，長度128比特，也就是16 字節。  
文件路徑長度上限是256 字節，假設平均長度是128 字節。  
如果用鍊錶法來解決衝突，還需要存儲指針，指針佔用8 字節。  
所以散列表中每個數據單元就佔用152 字節(估算)。  
假設每台機器的內存2GB ，散列表的裝載因子為0.75 ，那一台機器可以給大約1000萬( 2GB*0.75/152 )張圖片構建散列表。  
所以對1 億張圖片構建索引，需要大約十幾台機器。  
合理的估算能更好地評估解決方案的可行性。


**<font  size=5>3.7 分佈式存儲</font>**  

面對海量數據，為了提高數據的讀取、寫入能力，一般都採用分佈式的方式來存儲數據，比如分佈式緩存，  
借用前面數據分片的思想，即通過哈希算法對數據取哈希值，然後對機器個數取模，這個最終值就是應該存儲的緩存機器編號。  
但是這樣存在一個問題，假如後期需要新增或下線機器，個數發生改變，會造成緩存數據全部失效。    
這時，需要一致性哈希算法，具體可以查看 聊聊一致性哈希、漫畫算法：什麼是一致性哈希？  
大概含義是：  
實際k台機器，但劃分時使用m作為個數（m遠大於k），取模時，使用m。  
當被分配到不存在的機器上時，自動劃分到下一個存在的機器中。  
當新增機器時，只需調整新增機器與新增機器下一個存在的機器之間的緩存數據即可。  
下線機器時，只需調整該機器與下一台機器之間的數據即可，實際中為了避免突然機器下線，往往會在每台機器中同步上一台機器數據。  
為了解決最後一台機器的數據問題，往往還會把哈希空間做成環狀結構。

**<font  size=5>4. Hash算法的python實現</font>**  


**<font  size=5>4.1 自定義哈希函數</font>**  
自定義哈希函數通常可利用除留餘數、移位、循環哈希、平方取中等方法。下面這個例子就是我自己定義的一個哈希函數，運用了取模運算和異或運算。

In [1]:
#自定義哈希函數
def my_hash(x):
    return (x % 7)^2

print(my_hash(1))
print(my_hash(2))
print(my_hash(3))
print(my_hash(4))

3
0
1
6


**<font  size=5>4.2 hash()函數</font>**  
在python中有內置的哈希函數hash()，返回一個對象（數字、字符串，不能直接用於list、set、dictionary）的哈希值。示例代碼如下：

In [2]:
print(hash(1))
print(hash(1.0))#相同的數值，不同的類型，哈希值是一樣的
print(hash("abc"))
print(hash("hello world!"))

1
1
5320890315331711112
-7649595428275435302


在運行時發現了一個現象：相同字符串在同一次運行時的哈希值是相同的，但是不同次運行的哈希值不同。這是由於Python的字符串  
hash算法有一個啟動時隨機生成secret prefix/suffix的機制，存在隨機化現象：對同一個字符串輸入，不同解釋器進程得到的hash結果可  
能不同。因此當需要做可重現可跨進程保持一致性的hash，需要用到hashlib模塊。

**<font  size=5>4.3 hashlib模塊</font>**  
hashlib提供了常見的摘要算法，如MD5，SHA1等等。示例代碼如下：


In [4]:
# 使用hashlib模塊
import hashlib
 
md5 = hashlib.md5()     # 應用MD5算法
data = "hello world"
md5.update(data.encode('utf-8'))
print(md5.hexdigest())

5eb63bbbe01eeed093cb22bb8f5acdc3


**<font  size=5>6.例題 </font>**  
使用hash優化leetcode1